# Here is where the modelling process takes place, there will be a few models made, and then the best one will be chosen

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_colwidth = 350
pd.options.display.max_columns = 999

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, LassoCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [2]:
tweets = pd.read_csv('data/focus.csv')
tweets.head(1)

,date,username,name,tweet,mentions,replies_count,retweets_count,likes_count,video,quote_url,tweet_char_length,tweet_word_count
0,2020-07-14,elonmusk,Elon Musk,Cute,['teslarati'],222,176,7883,0,NaN,4,1


In [3]:
tweets.shape 

(9286, 12)

In [4]:
tweets.head(2)

,date,username,name,tweet,mentions,replies_count,retweets_count,likes_count,video,quote_url,tweet_char_length,tweet_word_count
0,2020-07-14,elonmusk,Elon Musk,Cute,['teslarati'],222,176,7883,0,NaN,4,1
1,2020-07-14,elonmusk,Elon Musk,Wow,"['davidtayar5', 'teslalisa', 'stevehamel16', 'vincent13031925', 's_padival', 'barkmsmeagol', 'annerajb', 'pjhornak', 'wpipperger', 'ecoheliguy']",346,180,4241,0,NaN,3,1


In [5]:
tweets.describe()

,replies_count,retweets_count,likes_count,video,tweet_char_length,tweet_word_count
count,9286.000000,9286.000000,9.286000e+03,9286.000000,9286.000000,9286.000000
mean,512.958432,2282.844066,1.873480e+04,0.007538,76.551045,12.217209
std,1720.320306,10546.775719,5.929579e+04,0.086500,67.004080,10.883635
min,0.000000,0.000000,2.100000e+01,0.000000,1.000000,1.000000
25%,43.250000,43.000000,9.430000e+02,0.000000,21.000000,3.000000
50%,103.000000,129.000000,2.341500e+03,0.000000,59.000000,9.000000
75%,371.000000,955.750000,1.159850e+04,0.000000,119.000000,19.000000
max,49529.000000,384289.000000,1.682551e+06,1.000000,447.000000,58.000000


In [6]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9286 entries, 0 to 9285
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date               9286 non-null   object
 1   username           9286 non-null   object
 2   name               9286 non-null   object
 3   tweet              9286 non-null   object
 4   mentions           9286 non-null   object
 5   replies_count      9286 non-null   int64 
 6   retweets_count     9286 non-null   int64 
 7   likes_count        9286 non-null   int64 
 8   video              9286 non-null   int64 
 9   quote_url          355 non-null    object
 10  tweet_char_length  9286 non-null   int64 
 11  tweet_word_count   9286 non-null   int64 
dtypes: int64(6), object(6)
memory usage: 870.7+ KB


In [7]:
sns.pairplot(tweets, corner=True)

## Count Vectorize the tweets to be used for modeling

In [8]:
cv = CountVectorizer()

In [9]:
tweets_cv = cv.fit_transform(tweets['tweet'])

In [10]:
tweet_cv_df = pd.DataFrame(tweets_cv.todense(), columns=cv.get_feature_names())
tweet_cv_df.head()

,00,000,0007,000th,01,0130,02,0202,02b2ooly,03,0387955011,03m,04,04727546592196,05,0525558616,06,0606,07,075,07pm8zy0xji,08,09,0aji,0di88eoz2c,0f0733ad841e,0lkwipnopb,0q,0qo78r_yyfa,0ujwqqpwmsy,0vg4wnuwvze6pbojdtq99k,0wopde7oe1k,10,100,1000,1000000,100011516,1000440472072962048,1000450938220376064,1000470928067776512,1000km,1000th,1000x,1001486475744063488,1001845642615017472,1001858174784172032,1001860891317387264,1001867688413544448,1001887654294773760,1002110600363872257,1002225801427935233,1002244316087992320,1002261751025491968,1002379548804206592,1007635225675993089,1009186716593393664,1009212679876202496,1009590475840466944,100b,100d,100k,100m,100mt,100mwh,100t,100x,101,1010191063640956928,1011081640158945280,1011796674283372544,1012783502004137984,1014221135850827776,1014293302592471042,1014515503740567553,1015811558851661825,1016966736946581504,1017206234439716864,1017312055605497860,1018319251428577280,1019589925358964744,1020012323606487041,1020490107546030080,1021225570766639104,1021810766712909824,1022995840363913216,1023774002915893248,1024335091785289729,10261,1026199248985972736,1026599623396192256,1026912973120462848,102800448,1028673766090399744,1030627360993947648,1030662218138890240,1031921389220098049,1032706860359417856,1032818998243520512,1033167462986997760,1037484104915345410,1038523411922731010,1040057735616888833,1040302986730209281,1041562562607960069,1041825248872681473,1041865950633418753,1045805476812746752,1047614884660240386,1047854644465164288,1048112638402674688,1049205935388811265,1049284924321087488,105,1050771311600041985,1051022913,1051865515260608512,1051878668136517632,1051904869441912832,1051911793373638661,1053390822991790083,1054488079208112133,1057718047102918656,1057765681108725760,105f,1061412788113825792,1062255003052769281,1063466830277369857,1063855556534898689,1064524502321778688,1067168653718814720,1067201940352786432,1067530874491715584,1069044263944511488,1069675820770222080,1070377864611033088,1071174335778447361,1072485984628105219,1074028244813279232,1075002416830054402,1076888923790553088,1078020855689547776,1080628997028040705,1081405732107370497,1081567268901928961,1081937517971193858,1082710262665355266,1082715123398062080,1083648618576281600,1084072568708124672,1084966116777091072,1087848110611722240,1089,1089354757188120578,1089937002038870018,1090625687826956288,1094019516944736256,1094026428264341504,1095085979419832320,1098762992848535553,1099037391669653509,10aac3a,10b,10cm,10k,10km,10m,10ms,10mw,10t,10th,10x,10y0vgjxgnq69,11,1100c,1101721765707825152,1101731517045268481,1101946598,1103428015030181888,1103760956830691328,1104032250495004673,1106401863535075328,1106429722378956801,1109495034959675393,110k,1110574284076769280,1110577741102022658,1110649850188558336,1112702954719588357,1112860196043452417,1113580660055101440,1113606676274384903,1113870913546670080,1114210401888100352,1114328048915099649,1115986443501838337,1116451957844193280,1116485160457793536,1118614166221271046,1119767408556687361,1119828529422770177,1120466365834514432,1120846381378224128,1123037242807603200,1124811199332225024,1125190944519786496,1125389151581409280,1125823642158243841,1126483572708249600,1126911564286767106,1127273426333388801,1129127540998705157,1129435468452417536,1129458434624053248,1129594966588547072,1131472501601644547,1131745938735681543,1131747570051686400,1131748543373271042,1131809805876654080,1132219856714194945,1132361490688761856,1136065718250643456,1138102943822098432,1138807297889787904,1138814698969677829,1139779810069114880,1140352388752130048,114405239,1145187380715855873,1146151371571380225,1146546495241371649,1147877818191106053,1151857907497132034,1151970082354028544,1153086461681422337,11587326607,11587930911,1158878301936484353,11591365620,11593729664,1159494489171513345,1159828206406553600,1160396658003329024,11605785,1161614701656252417,1163198089328390145,1166120736538288129,1166596870438866944,1167410401878974465,1170381150847741952,11711460841540

In [11]:
tweet_cv_df.sum(axis=0).sort_values(ascending=False).head(10)

to      2955
the     2648
is      2124
of      1931
in      1577
com     1496
for     1328
it      1264
will    1083
that    1083
dtype: int64

In [12]:
tweet_cv_df.sum(axis=0).sort_values(ascending=False).tail(10)

manual               1
manufacturability    1
manufactured         1
manufacturers        1
mapping              1
marcushousegame      1
margaritaville       1
marginal             1
margins              1
jacklipstone         1
dtype: int64

## Include a command to remove stop words, or words that are very common for basic communication

In [13]:
cv = CountVectorizer(stop_words=('english'))
tweets_cv = cv.fit_transform(tweets['tweet'])
tweet_cv_df = pd.DataFrame(tweets_cv.todense(), columns=cv.get_feature_names())
tweet_cv_df.sum(axis=0).sort_values(ascending=False).head(10)

com        1496
tesla      1030
twitter     977
https       867
pic         564
yes         464
just        392
status      373
good        363
model       348
dtype: int64

In [14]:
tweet_cv_df.shape

(9286, 12496)

## Use same command, but also use one to grab a number of grouped words related to each other.

In [15]:
cv = CountVectorizer(stop_words=('english'), ngram_range=(1, 3))
tweets_cv = cv.fit_transform(tweets['tweet'])
tweet_cv_df = pd.DataFrame(tweets_cv.todense(), columns=cv.get_feature_names())
tweet_cv_df.sum(axis=0).sort_values(ascending=False).head(10)

com                1496
tesla              1030
twitter             977
twitter com         917
https               867
pic                 564
pic twitter         544
pic twitter com     544
yes                 464
just                392
dtype: int64

In [16]:
tweet_cv_df.head()

,00,00 10aac3a,00 pic,00 pic twitter,000,000 000,000 000 year,000 000th,000 000th car,000 24,000 24 hours,000 caps,000 caps https,000 deaths,000 deaths tesla,000 hats,000 hats sold,000 horsepower,000 horsepower engine,000 jobs,000 jobs people,000 km,000 km 30,000 km apogee,000 km decent,000 km earth,000 km falcon,000 lb,000 lb 15,000 lb towing,000 lbs,000 lbs muck,000 lbs puny,000 left,000 miles,000 miles black,000 model,000 model cars,000 mph,000 mph like,000 ordered,000 ordered 19,000 people,000 people died,000 people directly,000 people hour,000 people indirectly,000 people massive,000 people worst,000 pic,000 pic twitter,000 projects,000 projects underway,000 psi,000 road,000 road legal,000 shaft,000 shaft horsepower,000 tesla,000 tesla cars,000 tesla smart,000 tons,000 tons thrust,000 tons year,000 year,000 year age,000 year generous,0007,0007 988648,0007 988648 covid,000th,000th car,000th car pic,01,01 07,01 07 electric,01 27,01 27 text,01 31,01 31 automakers,01 artificial,01 artificial intelligence,01 coming,01 coming soon,01 expensive,01 expensive build,01 fashion,01 fashion elon,01 secs,01 secs traction,01 weeks,01 weeks dec,0130,0130 c6ee,0130 c6ee 723c91aeae46,02,02 04,02 04 sure,02 10,02 10 tesla,02 11,02 11 consumer,02 20,02 20 tesla,02 27,02 27 bridgewater,02 27 spacex,02 ai,02 ai god,02 asia,02 asia china,02 autos,02 autos tesla,02 elon,02 elon musk,02 global,02 global vehicle,02 spacex,02 spacex fly,02 tesla,02 tesla earnings,0202,0202 story,0202 story html,02b2ooly,03,03 18,03 18 opinion,03 18 tesla,03 18 world,03 25,03 25 car,03 climate,03 climate change,03 dragon,03 dragon russian,03 tesla,03 tesla model,0387955011,0387955011 http,0387955011 http io9,03m,03m 100,03m 100 ms,04,04 05,04 05 feds,04 11,04 11 tesla,04 12,04 13,04 13 ftag,04 14,04 14 mystery,04 15,04 15 ventilators,04 20,04 20 ford,04 22,04 22 10,04 24,04 24 fact,04 25,04 25 tesla,04 28,04 28 texas,04 consumer,04 consumer reports,04 elon,04 elon musk,04 iss,04 iss recommends,04 little,04 little government,04 neuralink,04 neuralink html,04 short,04 short new,04 sure,04 sure looks,04 trust,04 trust computer,04727546592196,04727546592196 2c,04727546592196 2c 72,05,05 01,05 01 fashion,05 02,05 02 tesla,05 05,05 05 warren,05 06,05 06 love,05 12,05 12 18d2e740,05 18,05 18 video,05 21,05 21 fact,05 22,05 22 elon,05 23,05 23 movies,05 25,05 25 opinion,05 30,05 30 fact,05 31,05 31 elon,05 feds,05 feds raided,05 fermi,05 fermi paradox,05 india,05 india electric,05 south,05 south australias,05 warren,05 warren buffett,0525558616,06,06 04,06 04 elon,06 04 little,06 08,06 08 politics,06 12,06 12 opinion,06 13,06 13 tesla,06 18,06 18 tesla,06 20,06 20 science,06 22,06 22 6c2f7982,06 22 tesla,06 26,06 26 week,06 29,06 29 tesla,06 29 worlds,06 fortune,06 fortune elon,06 gates,06 gates says,06 health,06 health oumuamua,06 jay,06 jay leno,06 love,06 love tesla,0606,0606 4278,0606 4278 b454,07,07 02,07 02 asia,07 06,07 06 fortune,07 06 gates,07 07,07 07 tesla,07 10,07 10 german,07 11,07 11 tesla,07 14,07 14 european,07 28,07 28 elon,07 31,07 31 driving,07 electric,07 electric vehicles,07 fossil,07 fossil fuel,07 spacex,07 spacex steamroller,07 tesla,07 tesla model,075,075 remained,075 remained placed,07pm8zy0xji,08,08 06,08 06 jay,08 22,08 22 tesla,08 diff,08 diff 1st,08 politics,08 politics elon,08 spacex,08 spacex colonize,08 tesla,08 tesla model,09,09 09,09 09 simple,09 13,09 13 tesla,09 15,09 15 tesla,09 23,09 23 investors,09 drove,09 drove gm,09 simple,09 simple statistic,0aji,0di88eoz2c,0f0733ad841e,0f0733ad841e html,0lkwipnopb,0q,0qo78r_yyfa,0ujwqqpwmsy,0ujwqqpwmsy feature,0ujwqqpwmsy feature youtu,0vg4wnuwvze6pbojdtq99k,0vg4wnuwvze6pbojdtq99k si,0vg4wnuwvze6pbojdtq99k si zfnoubq9qrca4yv2tlirsg,0wopde7oe1k,10,10 000,10 000 km,10 000 year,10 02,10 02 ai,10 02 autos,10 03,10 03 tesla,10 1089,10 1089 ast,10 1089 space,10 14,10 14 days,10 15,10 15 mins,10 17,10 17 775,10 17 watch,10 20,10 20 deep,10 20 overly,10 20 years,10 200k,10 200k angel,10 21,10 21 pge,10 2

## Set up the training and testing sets based on what We are looking to predict

In [17]:
# X = tweets[['tweet', 'video', 'quote_url', 'tweet_char_length', 'tweet_word_count']]
X = tweets['tweet']
y1 = tweets['replies_count']

In [18]:
y2 = tweets['retweets_count']

In [19]:
y3 = tweets['likes_count']

In [20]:
X_train, X_test, y_train1, y_test1 = train_test_split(X, y1,
                                                    random_state=42,
                                                    test_size=.27)

In [21]:
X_train, X_test, y_train2, y_test2 = train_test_split(X, y2,
                                                    random_state=42,
                                                    test_size=.27)

In [22]:
X_train, X_test, y_train3, y_test3 = train_test_split(X, y3,
                                                    random_state=42,
                                                    test_size=.27)

In [23]:
X_train_cv = cv.fit(X_train)

In [24]:
X_train_cv = cv.transform(X_train)
X_test_cv = cv.transform(X_test)

In [25]:
X_train_cv.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [26]:
pd.DataFrame(X_train_cv.todense(), columns=cv.get_feature_names())

00  00 10aac3a  00 pic  00 pic twitter  000  000 000  000 000 year  \
0      0           0       0               0    0        0             0   
1      0           0       0               0    0        0             0   
2      0           0       0               0    0        0             0   
3      0           0       0               0    0        0             0   
4      0           0       0               0    0        0             0   
...   ..         ...     ...             ...  ...      ...           ...   
6773   0           0       0               0    0        0             0   
6774   0           0       0               0    0        0             0   
6775   0           0       0               0    0        0             0   
6776   0           0       0               0    0        0             0   
6777   0           0       0               0    0        0             0   

      000 000th  000 000th car  000 24  000 24 hours  000 caps  \
0             0              0       0             0         0   
1             0              0       0             0         0   
2             0              0       0             0         0   
3             0              0       0             0         0   
4             0              0       0             0         0   
...         ...            ...     ...           ...       ...   
6773          0              0       0             0         0   
6774          0              0       0             0         0   
6775          0              0       0             0         0   
6776          0              0       0             0         0   
6777          0              0       0             0         0   

      000 caps https  000 deaths  000 deaths tesla  000 hats  000 hats sold  \
0                  0           0                 0         0              0   
1                  0           0                 0         0              0   
2                  0           0                 0         0              0   
3                  0           0                 0         0              0   
4                  0           0                 0         0              0   
...              ...         ...               ...       ...            ...   
6773               0           0                 0         0              0   
6774               0           0                 0         0              0   
6775               0           0                 0         0              0   
6776               0           0                 0         0              0   
6777               0           0                 0         0              0   

      000 horsepower  000 horsepower engine  000 km  000 km apogee  \
0                  0                      0       0              0   
1                  0                      0       0              0   
2                  0                      0       0              0   
3                  0                      0       0              0   
4                  0                      0       0              0   
...              ...                    ...     ...            ...   
6773               0                      0       0              0   
6774               0                      0       0              0   
6775               0                      0       0              0   
6776               0                      0       0              0   
6777               0                      0       0              0   

      000 km earth  000 km falcon  000 lb  000 lb 15  000 lb towing  000 lbs  \
0                0              0       0          0              0        0   
1                0              0       0          0              0        0   
2                0              0       0          0              0        0   
3                0              0       0          0              0        0   
4                0              0       0          0              0        0   
...            ...            

In [27]:
X_train_df = pd.DataFrame(X_train_cv.todense(), columns=cv.get_feature_names())
X_train_df

00  00 10aac3a  00 pic  00 pic twitter  000  000 000  000 000 year  \
0      0           0       0               0    0        0             0   
1      0           0       0               0    0        0             0   
2      0           0       0               0    0        0             0   
3      0           0       0               0    0        0             0   
4      0           0       0               0    0        0             0   
...   ..         ...     ...             ...  ...      ...           ...   
6773   0           0       0               0    0        0             0   
6774   0           0       0               0    0        0             0   
6775   0           0       0               0    0        0             0   
6776   0           0       0               0    0        0             0   
6777   0           0       0               0    0        0             0   

      000 000th  000 000th car  000 24  000 24 hours  000 caps  \
0             0              0       0             0         0   
1             0              0       0             0         0   
2             0              0       0             0         0   
3             0              0       0             0         0   
4             0              0       0             0         0   
...         ...            ...     ...           ...       ...   
6773          0              0       0             0         0   
6774          0              0       0             0         0   
6775          0              0       0             0         0   
6776          0              0       0             0         0   
6777          0              0       0             0         0   

      000 caps https  000 deaths  000 deaths tesla  000 hats  000 hats sold  \
0                  0           0                 0         0              0   
1                  0           0                 0         0              0   
2                  0           0                 0         0              0   
3                  0           0                 0         0              0   
4                  0           0                 0         0              0   
...              ...         ...               ...       ...            ...   
6773               0           0                 0         0              0   
6774               0           0                 0         0              0   
6775               0           0                 0         0              0   
6776               0           0                 0         0              0   
6777               0           0                 0         0              0   

      000 horsepower  000 horsepower engine  000 km  000 km apogee  \
0                  0                      0       0              0   
1                  0                      0       0              0   
2                  0                      0       0              0   
3                  0                      0       0              0   
4                  0                      0       0              0   
...              ...                    ...     ...            ...   
6773               0                      0       0              0   
6774               0                      0       0              0   
6775               0                      0       0              0   
6776               0                      0       0              0   
6777               0                      0       0              0   

      000 km earth  000 km falcon  000 lb  000 lb 15  000 lb towing  000 lbs  \
0                0              0       0          0              0        0   
1                0              0       0          0              0        0   
2                0              0       0          0              0        0   
3                0              0       0          0              0        0   
4                0              0       0          0              0        0   
...            ...            

In [28]:
y_train1.value_counts(normalize=True)

26      0.010328
19      0.008557
22      0.008262
35      0.008114
28      0.007967
          ...   
4532    0.000148
430     0.000148
2475    0.000148
426     0.000148
2041    0.000148
Name: replies_count, Length: 1505, dtype: float64

In [29]:
y_train2.value_counts(normalize=True)

28       0.009590
29       0.009295
25       0.009147
31       0.007967
22       0.007967
           ...   
15045    0.000148
710      0.000148
2759     0.000148
714      0.000148
3960     0.000148
Name: retweets_count, Length: 2263, dtype: float64

In [30]:
y_train3.value_counts(normalize=True)

631       0.001180
841       0.001033
1025      0.001033
526       0.001033
774       0.000885
            ...   
7101      0.000148
127932    0.000148
11414     0.000148
5048      0.000148
40960     0.000148
Name: likes_count, Length: 4847, dtype: float64

In [31]:
mnb = MultinomialNB()
mnb.fit(X_train_cv, y_train1)

mnb.score(X_train_cv, y_train1), mnb.score(X_test_cv, y_test1)

(0.7384184125110652, 0.0035885167464114833)

## create multiple lists of stopwords to gridsearch through, unltimately make pipeline, or two

# Modeling for Replies Counts

In [75]:
la = Lasso()
la.fit(X_train_cv, y_train1)
la.score(X_train_cv, y_train1), la.score(X_test_cv, y_test1)

(0.651765945990536, -0.08129786249305271)

In [33]:
lcv = LassoCV()
lcv.fit(X_train_cv, y_train1)
lcv.score(X_train_cv, y_train1), lcv.score(X_test_cv, y_test1)

In [34]:
pipe1 = Pipeline([
    ('cv', CountVectorizer(min_df=5, max_features=1000)),
    ('lcv', LassoCV())
])


pipe1.fit(X_train, y_train1)
pipe1.score(X_train, y_train1), pipe1.score(X_test, y_test1)

In [35]:
pipe1.named_steps['lcv'].alpha_

In [36]:
pipe1.named_steps['lcv'].alphas_

# Modeling for Retweets Counts

In [37]:
la = Lasso()
la.fit(X_train_cv, y_train2)
la.score(X_train_cv, y_train2), la.score(X_test_cv, y_test2)

In [38]:
lcv = LassoCV()
lcv.fit(X_train_cv, y_train2)
lcv.score(X_train_cv, y_train2), lcv.score(X_test_cv, y_test2)

In [39]:
pipe2 = Pipeline([
    ('cv', CountVectorizer(min_df=5, max_features=1000)),
    ('lcv', LassoCV())
])


pipe2.fit(X_train, y_train2)
pipe2.score(X_train, y_train2), pipe2.score(X_test, y_test2)

In [40]:
pipe2.named_steps['lcv'].alpha_

In [41]:
pipe2.named_steps['lcv'].alphas_

In [42]:
pipe3 = Pipeline([
    ('cv', CountVectorizer(min_df=5, max_features=2500)),
    ('lcv', LassoCV())
])


pipe3.fit(X_train, y_train2)
pipe3.score(X_train, y_train2), pipe3.score(X_test, y_test2)

In [43]:
pipe3.named_steps['lcv'].alpha_

In [44]:
pipe3.named_steps['lcv'].alphas_

# Modeling for Likes Counts

In [45]:
la = Lasso()
la.fit(X_train_cv, y_train3)
la.score(X_train_cv, y_train3), la.score(X_test_cv, y_test3)

In [46]:
lcv = LassoCV()
lcv.fit(X_train_cv, y_train3)
lcv.score(X_train_cv, y_train3), lcv.score(X_test_cv, y_test3)

In [47]:
pipe4 = Pipeline([
    ('cv', CountVectorizer(min_df=5, max_features=1000)),
    ('lcv', LassoCV())
])


pipe4.fit(X_train, y_train3)
pipe4.score(X_train, y_train3), pipe4.score(X_test, y_test3)

In [48]:
pipe4.named_steps['lcv'].alpha_

In [49]:
pipe4.named_steps['lcv'].alphas_

In [50]:
pipe = Pipeline([
    ('cv', CountVectorizer()),
    ('lcv', LassoCV())
])

pipe_params = {
    'cv__stop_words': ['english', None],
    'cv__max_features': [1000, 2000, 3000, 4000, 5000, None],
    'cv__min_df': [1, 2, 3, 10, 15, 20, 30, 100],
    'cv__max_df': [.90, .95, 1.0],
    'cv__ngram_range': [(1,1), (1,2), (1,3)]
}

In [51]:
pipe = Pipeline([
    ('cv', CountVectorizer()),
    ('lcv', LassoCV())
])
pipe_params = {
    'cv__stop_words': ['english', None],
    'cv__max_features': [2000, 4000, None],
    'cv__min_df': [1, 2, 3, None],
    'cv__max_df': [.95, 1.0],
    'cv__ngram_range': [(1,1)]
}

In [63]:
pipe = Pipeline([
     ('cv', CountVectorizer()),
     ('lcv', LassoCV())
 ])
pipe_params = {
     'cv__stop_words': ['english', None],
     'cv__max_features': [4000, None],
     'cv__min_df': [2, 3],
     'cv__max_df': [.95, 1.0],
     'cv__ngram_range': [(1,1)]
 } 

In [68]:
pipe_replies = Pipeline([
    ('cv', CountVectorizer()),
    ('lcv', LassoCV())
])

gs_replies = GridSearchCV(pipe_replies,
                  pipe_params,
                  cv=3, n_jobs=-1, verbose=2)

In [69]:
pipe_retweets = Pipeline([
    ('cv', CountVectorizer()),
    ('lcv', LassoCV())
])

gs_retweets = GridSearchCV(pipe_retweets,
                  pipe_params,
                  cv=3, n_jobs=-1, verbose=2)

In [70]:
pipe_likes = Pipeline([
    ('cv', CountVectorizer()),
    ('lcv', LassoCV())
])

gs_likes = GridSearchCV(pipe_likes,
                  pipe_params,
                  cv=3, n_jobs=-1, verbose=2)

In [71]:
gs_replies.fit(X_train, y_train1)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 39.4min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 69.8min finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('cv', CountVectorizer()),
                                       ('lcv', LassoCV())]),
             n_jobs=-1,
             param_grid={'cv__max_df': [0.95, 1.0],
                         'cv__max_features': [4000, None], 'cv__min_df': [2, 3],
                         'cv__ngram_range': [(1, 1)],
                         'cv__stop_words': ['english', None]},
             verbose=2)

In [72]:
gs_replies.best_params_

{'cv__max_df': 0.95,
 'cv__max_features': 4000,
 'cv__min_df': 2,
 'cv__ngram_range': (1, 1),
 'cv__stop_words': None}

In [74]:
gs_replies.score(X_train, y_train1), gs_replies.score(X_test, y_test1)

(0.04726413716260747, 0.02145279170148262)

In [ ]:
gs_retweets.fit(X_train, y_train2)

In [ ]:
gs_retweets.best_params_

In [ ]:
gs_retweets.score(X_train, y_train2), gs_retweets.score(X_test, y_test2)

In [ ]:
gs_likes.fit(X_train, y_train3)

In [ ]:
gs_likes.best_params_

In [ ]:
gs_likes.score(X_train, y_train3), gs_likes.score(X_test, y_test3)

In [102]:
tweets['likes_success'] = (tweets['likes_count'] >= 7_500)

In [103]:
tweets['replies_success'] = (tweets['replies_count'] >= 2500)

In [104]:
tweets['retweets_success'] = (tweets['retweets_count'] >= 2500)

In [105]:
tweets.head(1)

,date,username,name,tweet,mentions,replies_count,retweets_count,likes_count,video,quote_url,tweet_char_length,tweet_word_count,likes_success,replies_success,retweets_success
0,2020-07-14,elonmusk,Elon Musk,Cute,['teslarati'],222,176,7883,0,NaN,4,1,True,False,False


In [106]:
# Thank you Caroline
tweets['retweets_success'] = tweets['retweets_success'].astype(int)

In [107]:
tweets['replies_success'] = tweets['retweets_success'].astype(int)

In [108]:
tweets['likes_success'] = tweets['retweets_success'].astype(int)

In [109]:
tweets.to_csv('data/update_tweets.csv', index=False)